In [2]:

from webbrowser import get
import requests
import base64
import urllib
import openai 
import pandas as pd
from urllib.parse import urlparse
import os
from dotenv import load_dotenv 
import json

openai.api_key = os.getenv("OPENAI_API_KEY")


In [3]:
readme = ""
summary = {}
with open("../sample_repos/catfact_readme.md","r") as file:
    readme = file.read()

with open("../generated_summaries/summary.json","r") as file:
    summary = json.load(file)


In [8]:
highlevel_prompt = f"""You are a markdown documentation generator. 
Please generate an intro.md file to code repository description provided. You will be documenting
the developer api that provides cat facts.
 
Here is a dict of filepath:file summary for each file
{summary}

this intro should be a high level description of the api functionality, and list endpoints with short descriptions of their function.
These docs should be written with respect to the end user of the api, and not focus on how the api works
"""

In [9]:
highlevel_completion = openai.Completion.create(
                        model="text-davinci-003",
                        prompt = highlevel_prompt,
                        temperature = 0,
                        max_tokens = 500
            ).choices[0].text

highlevel_completion

'\n# Cat Facts API\n\nWelcome to the Cat Facts API! This API provides a way to access cat facts from anywhere in the world. With this API, you can get a random cat fact, get a specific cat fact by ID, and submit your own cat facts.\n\n## Endpoints\n\n### Get Random Cat Fact\n\nThis endpoint returns a random cat fact.\n\n**Endpoint:** `GET /api/facts/random`\n\n### Get Cat Fact by ID\n\nThis endpoint returns a cat fact with the specified ID.\n\n**Endpoint:** `GET /api/facts/:id`\n\n### Submit Cat Fact\n\nThis endpoint allows you to submit your own cat fact.\n\n**Endpoint:** `POST /api/facts`'

# Cat Facts API

Welcome to the Cat Facts API! This API provides a way to access cat facts from anywhere in the world. With this API, you can get a random cat fact, get a specific cat fact by ID, and submit your own cat facts.

## Endpoints

### Get Random Cat Fact
This endpoint returns a random cat fact

**Endpoint:** `GET /api/facts/random`
### Get Cat Fact by ID\
This endpoint returns a cat fact with the specified ID.

**Endpoint:** `GET /api/facts/:id`
### Submit Cat Fact
This endpoint allows you to submit your own cat fact.

**Endpoint:** `POST /api/facts`'

In [10]:
code_index = {}

with open("../cache/code_cache.json","r") as file:
    code_index = json.load(file)["cached_index"]

fact_endpoint = code_index["app/routes/fact.routes.js"]
fact_model = code_index["app/models/fact.js"]
user_model = code_index["app/models/user.js"]

In [15]:
prompt = f"""
Please write markdown docs for the endpoint(s) described in the following code. Format like so:

# Endpoint_Category Endpoints

## Endpoint1 Name

```
GET /endpoint1/path
```
Description:

Query Parameters: 

Example Reponse: 

## Endpoint2 Name

```
GET /endpoint2/path
```
Description:

Query Parameters: 

Example Response: 

endpoint: {fact_endpoint}
data model: {fact_model}
"""

In [16]:
endpoint_completion = openai.Completion.create(
                        model="text-davinci-003",
                        prompt = prompt,
                        temperature = 0,
                        max_tokens = 2048
            ).choices[0].text

md_text = endpoint_completion
with open("../generated_docs/nb_docs.md","w") as file:
    file.write(md_text)

In [35]:
endpoint_indexer_prompt = """
You are a api documentation planner
You will be given a code file which defines api endpoints. You will plan
a markdown file which documents these endpoints

1. Name the markdown file - be specific
2. Parse the file into a list of endpoints
3. For each endpoint, do the following:
    - name the endpoint
    - Give the string indices this code is in between within the file.
        - start_index: line where endpoint is first defined
        - end_index: last line defining endpoint behavior & returning response
    - Give the filepath to the data model this endpoint uses in its response

json output format:

{
    "md_file": "[placeholder].md",
    "endpoints":[
        {
            "name": "endpoint name",
            "line_indices": {"start_index":int, "end_index"int}, 
            "response_model": "data_model file name"
        }
        ...
    ]
}

code:

"""

In [36]:
endpoint_indexer_completion = openai.Completion.create(
                        model="text-davinci-003",
                        prompt = endpoint_indexer_prompt+fact_endpoint,
                        temperature = 0,
                        max_tokens = 2048
            ).choices[0].text

md_text = endpoint_indexer_completion
with open("../generated_docs/endpoint_index.md","w") as file:
    file.write(md_text)

In [39]:
endpoint_index = {
    "md_file": "api_endpoints.md",
    "endpoints":[
        {
            "name": "/",
            "line_indices": {"start_index": 8, "end_index": 16}, 
            "response_model": "./app/models/fact"
        },
        {
            "name": "/me",
            "line_indices": {"start_index": 18, "end_index": 28}, 
            "response_model": "./app/models/fact"
        },
        {
            "name": "/random",
            "line_indices": {"start_index": 30, "end_index": 43}, 
            "response_model": "./app/models/fact"
        },
        {
            "name": "/:factID",
            "line_indices": {"start_index": 45, "end_index": 54}, 
            "response_model": "./app/models/fact"
        },
        {
            "name": "/",
            "line_indices": {"start_index": 56, "end_index": 68}, 
            "response_model": "./app/models/fact"
        }
    ]
}

In [40]:
fact_lines = fact_endpoint.split("\n")
for endpoint in endpoint_index["endpoints"]:
    line_indices = endpoint["line_indices"]
    ep = fact_lines[line_indices["start_index"]:line_indices["end_index"]]
    print('----------------------------------')
    for line in ep:
        print(line)

----------------------------------

router.get('/', async(req, res) => {
    try {
        const facts = await Fact.find().limit(5);
        return res.status(200).json(facts);
    } catch (err) {
        return res.status(err.status || 400).json(err);
    }
----------------------------------
// Get submitted facts
router.get('/me', async(req, res) => {

    const animalType = req.query.animal_type ? req.query.animal_type.split(',') : ['cat'];

    try {
        const data = await Fact.find({
                user: req.user._id,
                type: { $in: animalType }
            })
----------------------------------
            .limit(10)

        return res.status(200).json(data);
    } catch (err) {
        return res.status(400).json(err);
    }
});

// Get a random fact
router.get('/random', logApiRequest, async(req, res) => {

    const animalType = req.query.animal_type ? req.query.animal_type.split(',') : ['cat'];
    const amount = req.query.amount;
--------------------------